# Type cleanup and Missing-value treatment study

This notebook focuses on: (1) inspecting and cleaning column types, and (2) a systematic study of missing-value patterns and imputation strategies.

It is intentionally self-contained: open and run cells in order. Adjust file paths if needed.

## Setup: imports and dataset path

In [ ]:
import sys
import os

module_path = os.path.abspath("../preprocessing")
sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from data_cleaning import MissingValueHandler, initial_cleanup

sns.set(style="whitegrid")
DATA_PATH = Path('..') / 'data' / 'cleaned_welddb.csv'
columns = [
    "C", "Si", "Mn", "S", "P", "Ni", "Cr", "Mo", "V", "Cu", "Co", "W",
    "O", "Ti", "N", "Al", "B", "Nb", "Sn", "As", "Sb",
    "Current", "Voltage", "AC_DC", "ElectrodePolarity", "HeatInput",
    "InterpassTemp", "WeldType", "PWHT_Temp", "PWHT_Time",
    "YieldStrength", "UTS", "Elongation", "ReductionArea",
    "CharpyTemp", "CharpyImpact", "Hardness", "FATT50", "PrimaryFerrite",
    "Ferrite2ndPhase", "AcicularFerrite", "Martensite", "FerriteCarbide",
    "WeldID"
]
df = pd.read_csv(DATA_PATH)  # load dataset

# Quick peek
df.head()

## 1. Clean up data types

We inspect dtypes and perform safe conversions: parse dates, coerce numeric strings with `pd.to_numeric(errors='coerce')`, convert low-cardinality object columns to `category`, and normalize boolean-like columns. Always check the number of NaNs introduced by coercion.

In [ ]:
df = initial_cleanup(df)
mvh = MissingValueHandler(df)

## 2. Missing values: summary statistics and diagnostics

Compute missing counts and percentages per column and per row, flag columns above thresholds, and produce a compact flagging table with suggested actions.

In [ ]:
mvh.show_missing_summary(top=20)
mvh.plot_missingness('bar')
mvh.plot_missingness('heatmap')

## 3. Imputation strategy

The imputation strategy is : Thus we can try to intute patterns MAR, MCAR and MNAR:
- **MCAR**: correlations in the heatmap are close to 0 everywhere
- **MAR**: strong correlations between patterns of missing data
- **MNAR**: difficult to prove directly

Thus:
- **Chemical elements** (C, Mn, Cr, etc.) might be **MAR** : Correlated missing data between elements: depends on the analytical protocol
- **Mechanical properties** (YieldStrength, UTS, Elongation, etc.) might be **MAR** as well: Data missing together, therefore related to the availability of mechanical tests.
- **Microstructure** (PrimaryFerrite, AcicularFerrite, FerriteCarbide) might be **MNAR**: Probably absent when the microstructure is complex or not measured (non-random).
- **Process parameters** (Voltage, Polarity, InterpassTemp) might be **MCAR** or quasi-MCAR: Little or no correlation with other patterns of missing data.


With :

```python
process_param_columns = ['Current', 'Voltage','AC_DC', 'ElectrodePolarity', 'HeatInput', 'InterpassTemp', 'WeldType', 'PWHT_Temp', 'PWHT_Time']
chem_cols = ['C', 'Si', 'Mn', 'S', 'P', 'Ni', 'Cr', 'Mo', 'V', 'Cu', 'Co', 'W', 'O','Ti', 'N', 'Al', 'B', 'Nb', 'Sn', 'As', 'Sb']
mech_cols = [
    'YieldStrength', 'UTS', 'Elongation', 'ReductionArea',
    'CharpyTemp', 'CharpyImpact', 'Hardness', 'FATT50'
]
micro_cols = [
    'PrimaryFerrite', 'Ferrite2ndPhase', 'AcicularFerrite',
    'Martensite', 'FerriteCarbide'
]

In [ ]:
# 2. Handle process parameters (mostly MCAR/quasi-MCAR)
process_param_columns = [
    'Current', 'Voltage','AC_DC', 'ElectrodePolarity',
    'HeatInput', 'InterpassTemp', 'WeldType', 'PWHT_Temp', 'PWHT_Time'
]
mvh.simple_impute_for_process_params(process_param_columns=process_param_columns)

# 3. Clean chemical columns (likely MAR)
chem_cols = [
    'C', 'Si', 'Mn', 'S', 'P', 'Ni', 'Cr', 'Mo', 'V',
    'Cu', 'Co', 'W', 'O','Ti', 'N', 'Al', 'B', 'Nb', 'Sn', 'As', 'Sb'
]
mvh.clean_chemical_columns(chem_cols=chem_cols)
mvh.iterative_impute_columns(columns=chem_cols, max_iter=10)

# # 4. Handle mechanical properties (likely MAR)
mech_cols = [
    'UTS', 'Elongation', 'ReductionArea', # Do not include YieldStrength
    'CharpyTemp', 'CharpyImpact', 'Hardness', 'FATT50'
]
mvh.clean_and_impute_mechanical(mech_cols=mech_cols, create_flag=True, max_iter=10)

# # 5. Handle microstructure (likely MNAR)
micro_cols = [
    'PrimaryFerrite', 'Ferrite2ndPhase', 'AcicularFerrite',
    'Martensite', 'FerriteCarbide'
]
# Create MNAR indicators
mvh.mark_mnar(columns=micro_cols)

# # 6. Optional: drop rows/columns exceeding missingness thresholds
# mvh.drop_by_threshold(col_thresh=0.5, row_thresh=0.5)


In [ ]:
# Quick evaluation of imputation strategies
# We'll mask a fraction of observed values and measure imputation quality

# Choose a subset of columns to evaluate
eval_cols = chem_cols[:5] + process_param_columns[:3]

# Use 10% masking for testing
mask_fraction = 0.1

# Evaluate median imputation for numeric and mode for categorical
eval_results = mvh.evaluate_imputation(
    holdout_cols=eval_cols,
    mask_frac=mask_fraction,
    strategy='median'  # 'median' for numeric, mode internally for categorical
)

# Show results
print("Imputation evaluation results:")
print(eval_results)


**Quick snapshot of imputation performance**:

Chemical elements (C, Si, Mn, S, P): Very low MSE, so median imputation works well for these. Only Mn is a bit higher (~0.16), suggesting more variability or outliers.

Process parameters (Current, Voltage, AC_DC): Current has huge MSE (~59k) — maybe strong variability or missing patterns. Voltage is fine (~198), AC_DC (categorical) has high accuracy (0.976), so mode imputation works well.

**Takeaways**:

For chemicals, simple median works fine.

For process parameters, check if Current has extreme outliers — maybe use KNN or domain-specific imputation.

In [ ]:
timestamp = datetime.now().strftime("%Y_%m_%d")
mvh.save_dataframe(f"../data/data_imputed_{timestamp}.csv", format='csv')